# Analysis of NJ public schools results in ELA and math grades 6-8.

<span style="color: red;">**If kernel can't connect to server again run command:**
*netsh winsock reset*<span>

<a id="TOC"></a> 
## Table of Contents
1. [Data sources and definitions](#data)
2. [Imports: modules](#modules)
3. [Read and prepare data](#read)
4. [Best middle schools by math](#best)
5. [Maps of middle schools by math results](#maps) 


<a id="data"></a> 
#### Data:
1. Data New Jersey Student Learning Assessments (NJSLA) results 2015-2023 for grades 6-8:
<br>State of New Jersey, Department of Education:
Statewide Assessment Reports
<br>https://www.nj.gov/education/assessment/results/reports/
2. NJ schools locations: NJGIN Open Data <br>
https://njogis-newjersey.opendata.arcgis.com/datasets/d8223610010a4c3887cfb88b904545ff/explore
3. School districts: NJGIN Open Data 
<br>https://njogis-newjersey.opendata.arcgis.com/datasets/ca144194df66491d83b8f8bf338e0172/explore

####  Performance levels for New Jersey Student Learning Standards for English Language Arts and Math  

**Level 1**: Did Not Yet Meet Expectations 

**Level 2**: Partially Met Expectations 

**Level 3**: Approached Expectations  

**Level 4**: Met Expectations  

**Level 5**: Exceeded Expectations  

*Source: New Jersey Assessments Resource Center, 2022, https://nj.mypearsonsupport.com/resources/reporting/NJSLA_Score_Interpretation_Guide_Spring2022.pdf*

## Questions
*1. How the test results changed?*
<br>Compare last year test results in a school with the school 10-year average as percentage of average:
<br> school_change = (school_current_year - school_10year_average)/school_10year_average
<br> citywide_change = (city_current_year - city_10year_average)/city_10year_average
<br> relative_school_change = school_change - citywide_change
<br><br>
*2. How good the school is?* 
<br>Last three testing period results (2019, 2022, 2023) are different for some schools: due to COVID disruptions, testing procedures changes, in Destrict 15 due to admission rules changed. Therefore average 10 years scores do not reflect well schools situation now. Results for these 3 last testing years are teken instead.
<br><br>
*3. Is the school citywide or borowide?*
<br><br>
*4. Diversity?*
<br><br>
*5. Size?*

<a id="modules"></a> 
#### Imports: modules

In [1]:
import os
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
from tqdm import tqdm

pd.set_option('display.float_format', '{:.3f}'.format)

In [2]:
basePath = r"G:\My Drive\Kids\NJ_schools_mapped"
dataFolder = r"raw_data"
outputFolder = r"processed_data"

<a id="maps"></a> 
### Maps of the NYC middle schools by assessment results 

Below mapping is done with the __[Folium library](https://python-visualization.github.io/folium/latest/)__

#### Read schools geolocation file

### Preparing layer with districts

### Prepare schools layer

In [3]:
# Read saved GeoJSON with average tests results and plots
schoolsFile = 'NJpublicSchoolsData.geojson'
schoolsPath = os.path.join(basePath, outputFolder,schoolsFile)
schoolsData = gpd.read_file(schoolsPath)

In [4]:
schoolsData.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   OBJECTID                 1499 non-null   int64   
 1   DIST_CODE                1499 non-null   object  
 2   DIST_NAME                1499 non-null   object  
 3   SCHOOLCODE               1499 non-null   object  
 4   SCHOOLTYPE               1479 non-null   object  
 5   SCHOOL                   1499 non-null   object  
 6   SCHOOLNAME               1499 non-null   object  
 7   CITY                     1499 non-null   object  
 8   School Name              1499 non-null   object  
 9   School Code              1499 non-null   object  
 10  8yrs avg Lvl 1 Math      1492 non-null   float64 
 11  8yrs avg Lvl 2 Math      1492 non-null   float64 
 12  8yrs avg Lvl 3 Math      1492 non-null   float64 
 13  8yrs avg Lvl 4 Math      1492 non-null   float64 
 14  

In [7]:
unique_values = schoolsData['SCHOOLTYPE'].unique()
print(unique_values)

['CHARTER ELEMENTARY & KINDERGARTEN (K-6)' 'MIDDLE SCHOOL (GRADES 5-8)'
 'ELEMENTARY (GRADES 2-4)' 'HIGH SCHOOL (GRADES 9-12)'
 'PRIMARY (K-GRADE 1)' 'GRADES K-12' 'GRADES K-3' 'MIDDLE (GRADES 7-9)'
 'ELEMENTARY (GRADES 4-6)' 'ELEMENTARY (K-GRADE 4)' 'ELEMENTARY (GRADE 5)'
 'MIDDLE SCHOOL (GRADES 6-8)' 'ELEMENTARY (K-GRADE 3)'
 'ELEMENTARY (K-GRADE 5)' 'GRADES K-8' 'HIGH SCHOOL (9-12)'
 'ELEMENTARY (K-3)' 'MIDDLE SCHOOL (4-8)' 'PRIMARY SCHOOL (1-3)'
 'GRADES 4-7' 'ELEMENTARY (K-GRADE 6)'
 'MIDDLE/HIGH SCHOOL (GRADES 7 - 12)'
 'CHARTER TECHNICAL SCHOOL OR INSTITUTE' 'CHARTER HIGH SCHOOL'
 'GRADES K-7' 'GRADES K-10' 'GRADES K-5' 'GRADES K-11' 'GRADES K-4'
 'ELEMENTARY' 'CHARTER MIDDLE/HIGH SCHOOL' 'HIGH SCHOOL' 'GRADES 4-6'
 'INTERMEDIATE (7-9)' 'HIGH SCHOOL (GRADES 10-12)'
 'ELEMENTARY (K-GRADE 2)' 'ELEMENTARY SCHOOL (K-GRADE 5)'
 'ELEMENTARY (GRADES 3-8)' 'HS, GRADES 9-12' 'ELEMENTARY SCHOOL'
 'MIDDLE SCHOOL & KINDERGARTEN' 'Intermediate School (3 - 6)'
 'Middle School (7-8)' 'LOWER SC

#### Producing NYC map

In [5]:
# Prepare legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 300px; height: 110px; 
                 border:1px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 ">
                   <div>The size of each circle reflects the sum of average shares of students scoring at the highest level (level 5) on New Jersey state ELA and math tests over the years 2015 to 2023. &nbsp; </div>
                   <br>
                   <div><i class="fa fa-circle" style="border:2px solid #06a6cf; color:white; border-radius:50%; display:inline-block;"></i><span style="margin-left: 5px;">public school </span></div>
                   <div><i class="fa fa-circle" style="border:2px solid #f0a607; color:white; border-radius:50%; display:inline-block;"></i><span style="margin-left: 5px;">charter school </span></div>
                   
      </div>
     '''

In [8]:
from IPython.core.display import display, HTML

display(HTML("<style>.output_scroll { height: auto !important; max-height: 1500px; }</style>"))

# Create a map object, centered at NYC
schools_map = folium.Map(location=[40.0842, -74.5357], zoom_start=9, tiles="OpenStreetMap")
   
# Add dataframes with coordinates and test results to the map

def my_style(x):
    level5 = x['properties']['8yrs avg Lvl 5 Math+Ela']
#     charter = x['properties']['SCHOOLTYPE']
    charter_alt = x['properties']['School Name']
    color = 'orange' if 'CHARTER' in (charter_alt or '') else 'blue'
#     color = '#f0a607' if 'CHARTER' in (charter_alt or '') else '#06a6cf' 
    if level5 is None:
        level5 = 0
    #print(level5)
    return {
        "radius": (level5)*700,
        "color": color        
    }  


# ## Adding the layer to the map
# districts = folium.Choropleth(
#     geo_data = NYCDistrictsData,
#     data = DistrictAllData[DistrictAllData['Year'] == 2023],
#     columns = ['District','# Level 4 Math+ELA'],
#     key_on = "feature.properties.school_dist",
#     fill_color = "BuPu",
#     fill_opacity = 0.5,
#     line_opacity=0.3,
#     nan_fill_color="white",
#     legend_name = 'Sum of percentages of middle school test takers with Level 4 result in Math and ELA, 2023',
#     popup = folium.GeoJsonPopup(fields=["school_dist", "Year", "# Level 1", "# Level 2", "# Level 3", "# Level 4"]), 
#     name = "School districts"
# ).add_to(schools_map)


# Function to create iframe for a given row
def create_iframe(row):    
    html =  '<strong>{0}:</strong> {1}<br>\
    <strong>{2}:</strong> {3}<br>\
    <strong>{4}:</strong> {5}<br>\
    <strong>{6}:</strong> {7}<br>\
    <strong>{8}:</strong> {9}<br>\
    <img src="data:image/png;base64,{10}"><br>\
    <img src="data:image/png;base64,{11}">'.format(
        'School Name', row['SCHOOLNAME'],
        'School Code', row['SCHOOLCODE'],
        'School District', row['DIST_NAME'],
        'Level 5 share Avg 2015-2023 Math', round(row['8yrs avg Lvl 5 Math'], 2), 
        'Level 5 share Avg 2015-2023 ELA', round(row['8yrs avg Lvl 5 ELA'], 2),
        row['plot Math'], row['plot ELA'])
    return folium.IFrame(html, width=500, height=450)

def create_popup(x):
    iframe = create_iframe(x)
    popup = folium.Popup(iframe)
    return popup

# Iterate over the GeoDataFrame and add a popup to each feature
for _, row in tqdm(schoolsData.iterrows(), total = len(schoolsData)):
    iframe = create_iframe(row)
        
    data = gpd.GeoDataFrame(row.to_frame().T, crs=schoolsData.crs)
    
    folium.GeoJson(
    data,
    marker = folium.Circle(radius=10, fill_color='green', fill_opacity=0, color="green", weight=4),
    popup = folium.Popup(iframe),
    style_function = my_style, 
    control = False    
    #zoom_on_click = True,    
).add_to(schools_map)    
        
#folium.LayerControl().add_to(MathTestMS_map)    

#Adding legend to the map:
# Add the HTML to the map using a feature group

schools_map.get_root().html.add_child(folium.Element(legend_html))
  
# # Display the map
# schools_map

# Save map to html
mfile = 'NJShoolsMap.html'
mpath = os.path.join(basePath, outputFolder, mfile)
print(f'Saving map to file {mpath} ...')
schools_map.save(mpath)
print("Saved.")     

del mpath, mfile

100%|██████████████████████████████████████████████████████████████████████████████| 1499/1499 [00:24<00:00, 60.53it/s]


Saving map to file G:\My Drive\Kids\NJ_schools_mapped\processed_data\NJShoolsMap.html ...
Saved.


#### Producing map zoomed on District 15

In [ ]:
from IPython.core.display import display, HTML

display(HTML("<style>.output_scroll { height: auto !important; max-height: 1500px; }</style>"))

# Create a map object, centered at NYC
MathTestMS_map = folium.Map(location=[40.666591, -73.995518], zoom_start=13, tiles="cartodb positron")
   
# Add dataframes with coordinates and test results to the map

def my_style(x):
    level4 = x['properties']['3yrs avg Lvl 4 Math+Ela']
    openTo = x['properties']['Open to']
    color = 'yellow' if openTo == 'Citywide'  else '#3862e0' if openTo == 'Brooklyn' else '#54B96D'
    weight = 2 if openTo == 'Citywide'  else 2 if openTo == 'Brooklyn' else 0.5
    if level4 is None:
        level4 = 0
    #print(level4)
    return {
        "radius": (level4)*150,
        "color": color,
        "weight": weight
    }  


## Adding the layer to the map
districts = folium.Choropleth(
    geo_data = NYCDistrictsData,
    data = DistrictAllData[DistrictAllData['Year'] == 2023],
    columns = ['District','# Level 4 Math+ELA'],
    key_on = "feature.properties.school_dist",
    fill_color = "BuPu",
    fill_opacity = 0.5,
    line_opacity=0.3,
    nan_fill_color="white",
    legend_name = 'Sum of percentages of middle school test takers with Level 4 result in Math and ELA, 2023',
    popup = folium.GeoJsonPopup(fields=["school_dist", "Year", "# Level 1", "# Level 2", "# Level 3", "# Level 4"]), 
    name = "School districts"
).add_to(MathTestMS_map)

# folium.GeoJson(
#     NYCpublicSchoolsMath_mappable[NYCpublicSchoolsMath_mappable['Year'] == 2023],
#     marker = folium.Circle(radius=10, fill_color='yellow', fill_opacity=1.0, color="orange", weight=0.5),
#     tooltip = folium.GeoJsonTooltip(fields=["LEGAL_NAME","RECORD_TYPE_DESC", "# Level 4"]),
#     popup = folium.GeoJsonPopup(fields=["LEGAL_NAME","RECORD_TYPE_DESC", "# Level 4"]),
#     style_function = my_style,        
#     zoom_on_click = True,
#     name = "All middle schools"
# ).add_to(MathTestMS_map)

# Function to create iframe for a given row
def create_iframe(row):    
    html =  '<strong>{0}:</strong> {1}<br><strong>{2}:</strong> {3}<br><strong>{4}:</strong> {5}<br>\
    <strong>{6}:</strong> {7}<br><strong>{8}:</strong> {9}<br><img src="data:image/png;base64,{10}"><br>\
    <img src="data:image/png;base64,{11}"<br> <img src="data:image/png;base64,{12}">'.format(
        'School Name', row['LEGAL_NAME'],
        'School Number', row['DBN'],
        'Total Enrollment, students', row['Total Enrollment'],
        'Level 4 share Avg 2019-2023 Math', round(row['3yrs avg Lvl 4 Math'], 2), 
        'Level 4 share Avg 2019-2023 ELA', round(row['3yrs avg Lvl 4 ELA'], 2),
        row['plot Math'], row['plot ELA'], row['Dvst_chart'])
    return folium.IFrame(html, width=500, height=450)

def create_popup(x):
    iframe = create_iframe(x)
    popup = folium.Popup(iframe)
    return popup

# Iterate over the GeoDataFrame and add a popup to each feature
for _, row in tqdm(NYCSchoolsAVGData.iterrows(), total = len(NYCSchoolsAVGData)):
    iframe = create_iframe(row)
        
    data = gpd.GeoDataFrame(row.to_frame().T, crs=NYCSchoolsAVGData.crs)
    
    folium.GeoJson(
    data,
    marker = folium.Circle(radius=10, fill_color='green', fill_opacity=1.0, color="green", weight=0.5),
    popup = folium.Popup(iframe),
    style_function = my_style, 
    control = False    
    #zoom_on_click = True,    
).add_to(MathTestMS_map)    
        
folium.LayerControl().add_to(MathTestMS_map)    
  
# # Display the map
# MathTestMS_map

# Save map to html
mfile = 'NYCMSmap_Dist15.html'
mpath = os.path.join(basePath, outputFolder, mfile)
print(f'Saving map to file {mpath} ...')
MathTestMS_map.save(mpath)
print("Saved.")   